# WEEK 4 Exercises - Making Queries

![ERD-E-COMMERCE](https://github.com/it-web-pro/django-week4/blob/main/images/WEEK3-ERD(e-commerce).png?raw=true)

## Instruction

1. สร้าง `virtual environment`
2. ติดตั้ง `django` และ `psycopg2` libraries
3. สร้างโปรเจคใหม่ใหม่ชื่อ`myshop`
4. จากนั้นให้ทำการ startapp ใหม่ชื่อ `shop`
5. สร้าง database ชื่อ `shop` ใน Postgres DB
6. ทำการเพิ่ม code ด้านล่างนี้ในไฟล์ `shop/models.py`
7. เพิ่ม **'shop'** ใน `settings.py`
8. ทำการ `makemigrations` และ `migrate`

```python
from django.db import models

# Create your models here.


class Customer(models.Model):
    first_name = models.CharField(max_length=150)
    last_name = models.CharField(max_length=200)
    email = models.CharField(max_length=150)
    address = models.JSONField(null=True)

class ProductCategory(models.Model):
    name = models.CharField(max_length=150)

class Product(models.Model):
    name = models.CharField(max_length=150)
    description = models.TextField(null=True, blank=True)
    remaining_amount = models.PositiveIntegerField(default=0)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    categories = models.ManyToManyField(ProductCategory)

class Cart(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    create_date = models.DateTimeField()
    expired_in = models.PositiveIntegerField(default=60)
    
class CartItem(models.Model):
    cart = models.ForeignKey(Cart, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Order(models.Model):
    customer = models.ForeignKey(Customer, on_delete=models.CASCADE)
    order_date = models.DateField()
    remark = models.TextField(null=True, blank=True)

class OrderItem(models.Model):
    order = models.ForeignKey(Order, on_delete=models.CASCADE)
    product = models.ForeignKey(Product, on_delete=models.CASCADE)
    amount = models.PositiveIntegerField(default=1)
    
class Payment(models.Model):
    order = models.OneToOneField(Order, on_delete=models.PROTECT)
    payment_date = models.DateField()
    remark = models.TextField(null=True, blank=True)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)

class PaymentItem(models.Model):
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    order_item = models.OneToOneField(OrderItem, on_delete=models.CASCADE)
    price = models.DecimalField(max_digits=10, decimal_places=2)
    discount = models.DecimalField(max_digits=10, decimal_places=2, default=0)
    
class PaymentMethod(models.Model):
    class MethodChoices(models.Choices):
        QR = "QR"
        CREDIT = "CREDIT"
    
    payment = models.ForeignKey(Payment, on_delete=models.CASCADE)
    method = models.CharField(max_length=15, choices=MethodChoices.choices)
    price = models.DecimalField(max_digits=10, decimal_places=2)
```

**จากนั้นให้ทำการ migrate และ run คำสั่งในไฟล์ `shop.sql` ใน PgAdmin4**

In [1]:
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

### 1. ให้นักศึกษา Query ค้นหาข้อมูลมาแสดงให้ถูกต้องตามโจทย์

1.1 query หาข้อมูล `Order` ทั้งหมดที่เกิดขึ้นในเดือน `พฤษภาคม` มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
ORDER ID:22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID:23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID:24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID:25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID:26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID:27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID:28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID:29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID:30, DATE: 2024-05-03, PRICE: 1690.00
ORDER ID:31, DATE: 2024-05-04, PRICE: 3280.00
```

In [25]:
from shop.models import Order, Payment
payment = Payment.objects.filter(order__order_date__month=5)[1:11]
for p in payment:
    print(f"ORDER ID: {p.order.id}, DATE:{p.order.order_date}, PRICE:{p.price}")

ORDER ID: 22, DATE:2024-05-01, PRICE:4890.00
ORDER ID: 23, DATE:2024-05-01, PRICE:2540.00
ORDER ID: 24, DATE:2024-05-01, PRICE:1720.00
ORDER ID: 25, DATE:2024-05-02, PRICE:322499.00
ORDER ID: 26, DATE:2024-05-02, PRICE:3399.00
ORDER ID: 27, DATE:2024-05-02, PRICE:1190.00
ORDER ID: 28, DATE:2024-05-03, PRICE:9499.00
ORDER ID: 29, DATE:2024-05-03, PRICE:700.00
ORDER ID: 30, DATE:2024-05-03, PRICE:1690.00
ORDER ID: 31, DATE:2024-05-04, PRICE:3280.00


In [25]:
from shop.models import Order, Payment
order = Order.objects.filter(order_date__month=5)
payment = Payment.objects.filter(order__order_date__month=5)[:10]
for ans in payment:
    print(f"ORDER ID: {ans.order.id}, DATE: {ans.order.order_date}, PRICE: {ans.price:.2f}")


ORDER ID: 22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID: 23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID: 24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID: 25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID: 26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID: 27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID: 28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID: 29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID: 30, DATE: 2024-05-03, PRICE: 1690.00
ORDER ID: 31, DATE: 2024-05-04, PRICE: 3280.00


1.2 query หาข้อมูล `Product` ที่มีคำลงท้ายว่า `features.` ในรายละเอียดสินค้า และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.
```

In [15]:
from shop.models import Product
product = Product.objects.filter(description__endswith='features.')
for ans in product:
    print(f"PRODUCT: {ans.id}, DESCRIPTION: {ans.description}")



PRODUCT: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT: 45, DESCRIPTION: Customizable racing track set with loop and jump features.


1.3 query หาข้อมูล `Product` ที่มีราคาสินค้าตั้งแต่ `5000.00` ขึ้นไป และอยู่ในหมวดหมู่ `Information Technology` และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 10, NAME: Smartwatch, PRICE: 4500.00
```

In [16]:
from shop.models import Product
product = Product.objects.filter(price__gte=5000.00,categories__name="Information Technology")
for ans in product:
    print(f"PRODUCT ID: {ans.id}, NAME: {ans.name}, PRICE: {ans.price:.2f}")



PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
PRODUCT ID: 70, NAME: Notebook HP Pavilion Silver, PRICE: 20000.00


1.4 query หาข้อมูล `Product` ที่มีราคาสินค้าน้อยกว่า `200.00` และมากกว่า `100.00` และแสดงผลดังตัวอย่าง (0.5 คะแนน)

```txt
PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 66, NAME: Salvage the Bones, PRICE: 129.00
```

In [10]:
from shop.models import Product
product = Product.objects.filter(price__gt=100.00, price__lt=200.00)
for ans in product:
    print(f"PRODUCT ID: {ans.id}, NAME: {ans.name}, PRICE: {ans.price:.2f}")


PRODUCT ID: 28, NAME: Women's Sweater, PRICE: 190.00
PRODUCT ID: 66, NAME: Salvage the Bones, PRICE: 129.00


### 2. เพิ่ม ลบ แก้ไข สินค้า

2.1 ให้เพิ่มสินค้าใหม่จำนวน 3 รายการ (0.5 คะแนน)

```txt
สินค้าที่ 1
ชื่อ: Philosopher's Stone (1997)
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 20
รายละเอียดซ: By J. K. Rowling.
ราคา: 790

สินค้าที่ 2
ชื่อ: Me Before You
หมวดหมู่สินค้า: Books and Media
จำนวนคงเหลือ: 40
รายละเอียดซ: A romance novel written by Jojo
ราคา: 390

สินค้าที่ 3
ชื่อ: Notebook HP Pavilion Silver
หมวดหมู่สินค้า: Information Technology และ Electronics
จำนวนคงเหลือ: 10
รายละเอียดซ: Display Screen. 16.0
ราคา: 20000
```

In [18]:

cat1 = ProductCategory.objects.get(name='Books and Media')
pd1 = Product(name="Philosopher's Stone (1997)",description="By J. K. Rowling.",remaining_amount=20,price=790.00)
pd2 = Product(name="Me Before You",description="A romance novel written by Jojo",remaining_amount=40,price=390.00)
pd1.save()
pd2.save()
pd1.categories.add(cat1)
pd2.categories.add(cat1)

cat1 = ProductCategory.objects.get(name="Information Technology")
cat2 = ProductCategory.objects.get(name='Electronics')
pd3 = Product( name="Notebook HP Pavilion Silver", description="Display Screen. 16.0",remaining_amount=10,price=20000.00)
pd3.save()
pd3.categories.add(cat1,cat2)


2.2 แก้ไขชื่อสินค้า จาก `Philosopher's Stone (1997)` เป็น `Half-Blood Prince (2005)` (0.5 คะแนน)

In [23]:
pd1.name = "Half-Blood Prince (2005)"
pd1.save()


2.3 แก้ไขชื่อหมวดหมู่สินค้า จาก `Books and Media` เป็น `Books` (0.5 คะแนน)


In [13]:
cat = ProductCategory.objects.get(name='Books and Media')
cat.name = "Book"
cat.save()

2.4 ลบสินค้าทุกตัวที่อยู่ในหมวดหมู่ `Books` (0.5 คะแนน)

In [7]:
Product.objects.filter(categories__name="Book").delete()

(10, {'shop.Product_categories': 5, 'shop.Product': 5})

In [19]:
# query หาข้อมูล Order ทั้งหมดที่เกิดขึ้นในเดือน พฤษภาคม มาแสดงผล 10 รายการแรก และแสดงผลดังตัวอย่าง 0.5 คะแนน 
from shop.models import Order, Payment
# Query ข้อมูล Payment ที่มี Order ที่เกิดขึ้นในเดือนพฤษภาคม และจำกัดผลลัพธ์ที่ 10 รายการแรก
payments = Payment.objects.filter(order__order_date__month=5)[:10]

# แสดงผลลัพธ์ตามตัวอย่าง
for payment in payments:
    print(f"ORDER ID: {payment.order.id}, DATE: {payment.order.order_date}, PRICE: {payment.price:.2f}")
    #payment.order.id คือการเข้าถึง id ของ Order
    #payment.order.order_date คือการเข้าถึง order_date ของ Order

ORDER ID: 60, DATE: 2024-05-13, PRICE: 600.00
ORDER ID: 22, DATE: 2024-05-01, PRICE: 4890.00
ORDER ID: 23, DATE: 2024-05-01, PRICE: 2540.00
ORDER ID: 24, DATE: 2024-05-01, PRICE: 1720.00
ORDER ID: 25, DATE: 2024-05-02, PRICE: 322499.00
ORDER ID: 26, DATE: 2024-05-02, PRICE: 3399.00
ORDER ID: 27, DATE: 2024-05-02, PRICE: 1190.00
ORDER ID: 28, DATE: 2024-05-03, PRICE: 9499.00
ORDER ID: 29, DATE: 2024-05-03, PRICE: 700.00
ORDER ID: 30, DATE: 2024-05-03, PRICE: 1690.00


In [ ]:
query หาข้อมูล Product ที่มีคำลงท้ายว่า features. ในรายละเอียดสินค้า และแสดงผลดังตัวอย่าง (0.5 คะแนน)

PRODUCT ID: 1, DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID: 7, DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID: 10, DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID: 14, DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID: 45, DESCRIPTION: Customizable racing track set with loop and jump features.

In [28]:
from shop.models import Product
product = Product.objects.filter(description__endswith='features.')
for p in product:
    print(f"PRODUCT ID:{p.id} ,DESCRIPTION: {p.description}")

PRODUCT ID:1 ,DESCRIPTION: A sleek and powerful smartphone with advanced features.
PRODUCT ID:7 ,DESCRIPTION: High-resolution digital camera with advanced photography features.
PRODUCT ID:10 ,DESCRIPTION: A stylish smartwatch with health monitoring and notification features.
PRODUCT ID:14 ,DESCRIPTION: Split air conditioner with remote control and energy-saving features.
PRODUCT ID:45 ,DESCRIPTION: Customizable racing track set with loop and jump features.


In [ ]:
query หาข้อมูล Product ที่มีราคาสินค้าตั้งแต่ 5000.00 ขึ้นไป และอยู่ในหมวดหมู่ Information Technology และแสดงผลดังตัวอย่าง (0.5 คะแนน)

PRODUCT ID: 1, NAME: Smartphone, PRICE: 5900.00
PRODUCT ID: 2, NAME: Laptop, PRICE: 25999.00
PRODUCT ID: 3, NAME: Smart TV, PRICE: 8900.00
PRODUCT ID: 5, NAME: Tablet, PRICE: 12900.00
PRODUCT ID: 6, NAME: Gaming Console, PRICE: 5000.00
PRODUCT ID: 7, NAME: Digital Camera, PRICE: 32000.00
 

In [32]:
from shop.models import Product
product = Product.objects.filter(price__gte=5000,categories__name='Information Technology')
for p in product:
    print(f"PRODUCT ID:{p.id}, Name:{p.name}")

PRODUCT ID:1, Name:Smartphone
PRODUCT ID:2, Name:Laptop
PRODUCT ID:3, Name:Smart TV
PRODUCT ID:5, Name:Tablet
PRODUCT ID:6, Name:Gaming Console
PRODUCT ID:7, Name:Digital Camera


In [ ]:
 Query หาข้อมูล Customer ที่มีอีเมลลงท้ายด้วย "example.com"

In [54]:
from shop.models import Customer
customer = Customer.objects.filter(email__endswith='.com')
for c in customer:
    print(f"CUSTOMER ID: {c.id}, NAME: {c.first_name} {c.last_name}, EMAIL: {c.email}")

CUSTOMER ID: 1, NAME: Panita Hongsakulpan, EMAIL: panita.hong@gmail.com
CUSTOMER ID: 2, NAME: Pakin Janpen, EMAIL: pakin.jan@gmail.com
CUSTOMER ID: 3, NAME: Jenjira Sukanansarn, EMAIL: jenjira.su@gmail.com
CUSTOMER ID: 4, NAME: Dejwit Tangjareonsakul, EMAIL: dejwit.tt@gmail.com
CUSTOMER ID: 5, NAME: Pong Sawadiwong, EMAIL: pong.23@gmail.com
CUSTOMER ID: 6, NAME: Thitirat Sukkesorn, EMAIL: thiti.za@gmail.com
CUSTOMER ID: 7, NAME: Porntipa Pasakul, EMAIL: prontipa.za@gmail.com
CUSTOMER ID: 8, NAME: Warit Pititat, EMAIL: warit.za@gmail.com
CUSTOMER ID: 9, NAME: Sira Pititat, EMAIL: sira.za@gmail.com
CUSTOMER ID: 10, NAME: Wanaporn Klabpetch, EMAIL: wanaporn.over@gmail.com
CUSTOMER ID: 11, NAME: Jack Maa, EMAIL: jack.maa@gmail.com
CUSTOMER ID: 12, NAME: Jakkapob Sopapak, EMAIL: jakkapob@gmail.com
CUSTOMER ID: 13, NAME: Kittitorn Wilairat, EMAIL: kittitorn@gmail.com
CUSTOMER ID: 14, NAME: Yanisa Nantapak, EMAIL: yanisa.ok@gmail.com
CUSTOMER ID: 15, NAME: stave klolas, EMAIL: mr.stave@gmail.

In [ ]:
 Query หาข้อมูล Cart ที่ถูกสร้างขึ้นในปี 2023

In [57]:
from shop.models import Cart
cart = Cart.objects.filter(create_date__year=2024)
for c in cart:
    print(f"CUSTOMER ID: {c.customer.id}")

CUSTOMER ID: 13
CUSTOMER ID: 30
CUSTOMER ID: 5
CUSTOMER ID: 12
CUSTOMER ID: 24
CUSTOMER ID: 21
CUSTOMER ID: 11
CUSTOMER ID: 30
CUSTOMER ID: 7
CUSTOMER ID: 4
CUSTOMER ID: 29
CUSTOMER ID: 9
CUSTOMER ID: 26
CUSTOMER ID: 6
CUSTOMER ID: 23
CUSTOMER ID: 7
CUSTOMER ID: 6
CUSTOMER ID: 16
CUSTOMER ID: 23
CUSTOMER ID: 21
CUSTOMER ID: 15
CUSTOMER ID: 26
CUSTOMER ID: 14
CUSTOMER ID: 30
CUSTOMER ID: 14
CUSTOMER ID: 13
CUSTOMER ID: 25
CUSTOMER ID: 4
CUSTOMER ID: 28
CUSTOMER ID: 2
CUSTOMER ID: 16
CUSTOMER ID: 7
CUSTOMER ID: 18
CUSTOMER ID: 16
CUSTOMER ID: 8
CUSTOMER ID: 12
CUSTOMER ID: 17
CUSTOMER ID: 9
CUSTOMER ID: 23
CUSTOMER ID: 29
CUSTOMER ID: 3
CUSTOMER ID: 27
CUSTOMER ID: 28
CUSTOMER ID: 8
CUSTOMER ID: 11
CUSTOMER ID: 24
CUSTOMER ID: 1
CUSTOMER ID: 5
CUSTOMER ID: 30
CUSTOMER ID: 3
CUSTOMER ID: 12
CUSTOMER ID: 6
CUSTOMER ID: 19
CUSTOMER ID: 11
CUSTOMER ID: 18
CUSTOMER ID: 24
CUSTOMER ID: 14
CUSTOMER ID: 22
CUSTOMER ID: 15
CUSTOMER ID: 14
CUSTOMER ID: 30
CUSTOMER ID: 26
CUSTOMER ID: 29
CUSTOMER I

In [ ]:
 Query หาข้อมูล OrderItem ที่มีจำนวนสินค้ามากกว่า 5

In [65]:
from shop.models import OrderItem
orderitem = OrderItem.objects.filter(amount__gte=1)
for c in orderitem:
    print(f"CUSTOMER ID: {c.order.customer.id}")

CUSTOMER ID: 5
CUSTOMER ID: 5
CUSTOMER ID: 3
CUSTOMER ID: 3
CUSTOMER ID: 2
CUSTOMER ID: 2
CUSTOMER ID: 28
CUSTOMER ID: 28
CUSTOMER ID: 12
CUSTOMER ID: 12
CUSTOMER ID: 5
CUSTOMER ID: 5
CUSTOMER ID: 4
CUSTOMER ID: 4
CUSTOMER ID: 29
CUSTOMER ID: 29
CUSTOMER ID: 17
CUSTOMER ID: 17
CUSTOMER ID: 27
CUSTOMER ID: 27
CUSTOMER ID: 30
CUSTOMER ID: 30
CUSTOMER ID: 4
CUSTOMER ID: 4
CUSTOMER ID: 3
CUSTOMER ID: 3
CUSTOMER ID: 19
CUSTOMER ID: 19
CUSTOMER ID: 23
CUSTOMER ID: 23
CUSTOMER ID: 30
CUSTOMER ID: 30
CUSTOMER ID: 21
CUSTOMER ID: 21
CUSTOMER ID: 26
CUSTOMER ID: 26
CUSTOMER ID: 21
CUSTOMER ID: 21
CUSTOMER ID: 30
CUSTOMER ID: 30
CUSTOMER ID: 20
CUSTOMER ID: 20
CUSTOMER ID: 23
CUSTOMER ID: 23
CUSTOMER ID: 8
CUSTOMER ID: 8
CUSTOMER ID: 24
CUSTOMER ID: 24
CUSTOMER ID: 20
CUSTOMER ID: 20
CUSTOMER ID: 7
CUSTOMER ID: 7
CUSTOMER ID: 25
CUSTOMER ID: 25
CUSTOMER ID: 4
CUSTOMER ID: 4
CUSTOMER ID: 3
CUSTOMER ID: 3
CUSTOMER ID: 30
CUSTOMER ID: 30
CUSTOMER ID: 17
CUSTOMER ID: 17
CUSTOMER ID: 22
CUSTOMER ID: 2

In [ ]:
 Query หาข้อมูล Payment ที่มีส่วนลดมากกว่า 100.00

In [74]:
payment = Payment.objects.filter(discount__gt=10000.00)
for c in payment:
    print(f"PAYMENT ID: {c.id}, ORDER ID: {c.order.id}, DISCOUNT: {c.discount:.2f}")

PAYMENT ID: 7, ORDER ID: 5, DISCOUNT: 113446.20
PAYMENT ID: 18, ORDER ID: 16, DISCOUNT: 121922.64
PAYMENT ID: 27, ORDER ID: 25, DISCOUNT: 33827.41
PAYMENT ID: 45, ORDER ID: 43, DISCOUNT: 126859.95
PAYMENT ID: 77, ORDER ID: 75, DISCOUNT: 19397.00
PAYMENT ID: 82, ORDER ID: 80, DISCOUNT: 46229.40
PAYMENT ID: 92, ORDER ID: 90, DISCOUNT: 29433.25
PAYMENT ID: 105, ORDER ID: 103, DISCOUNT: 105019.11
PAYMENT ID: 125, ORDER ID: 123, DISCOUNT: 17939.55
PAYMENT ID: 127, ORDER ID: 125, DISCOUNT: 14578.90
PAYMENT ID: 137, ORDER ID: 135, DISCOUNT: 71407.25
PAYMENT ID: 178, ORDER ID: 176, DISCOUNT: 28042.00
PAYMENT ID: 186, ORDER ID: 184, DISCOUNT: 138060.00


In [ ]:
Query หาข้อมูล Customer ที่ไม่มีที่อยู่ (address เป็น null)

In [81]:
cus = Customer.objects.filter(address__isnull=False)
for c in cus:
    print(f"CUSTOMER ID: {c.id}, NAME: {c.first_name} {c.last_name}, EMAIL: {c.email}")

CUSTOMER ID: 1, NAME: Panita Hongsakulpan, EMAIL: panita.hong@gmail.com
CUSTOMER ID: 2, NAME: Pakin Janpen, EMAIL: pakin.jan@gmail.com
CUSTOMER ID: 3, NAME: Jenjira Sukanansarn, EMAIL: jenjira.su@gmail.com
CUSTOMER ID: 4, NAME: Dejwit Tangjareonsakul, EMAIL: dejwit.tt@gmail.com
CUSTOMER ID: 5, NAME: Pong Sawadiwong, EMAIL: pong.23@gmail.com
CUSTOMER ID: 6, NAME: Thitirat Sukkesorn, EMAIL: thiti.za@gmail.com
CUSTOMER ID: 7, NAME: Porntipa Pasakul, EMAIL: prontipa.za@gmail.com
CUSTOMER ID: 8, NAME: Warit Pititat, EMAIL: warit.za@gmail.com
CUSTOMER ID: 9, NAME: Sira Pititat, EMAIL: sira.za@gmail.com
CUSTOMER ID: 10, NAME: Wanaporn Klabpetch, EMAIL: wanaporn.over@gmail.com
CUSTOMER ID: 11, NAME: Jack Maa, EMAIL: jack.maa@gmail.com
CUSTOMER ID: 12, NAME: Jakkapob Sopapak, EMAIL: jakkapob@gmail.com
CUSTOMER ID: 13, NAME: Kittitorn Wilairat, EMAIL: kittitorn@gmail.com
CUSTOMER ID: 14, NAME: Yanisa Nantapak, EMAIL: yanisa.ok@gmail.com
CUSTOMER ID: 15, NAME: stave klolas, EMAIL: mr.stave@gmail.

In [ ]:
 Query หาข้อมูล CartItem ที่มีสินค้าจากหมวดหมู่ "Sports"

In [88]:
cart = CartItem.objects.filter(product__categories__name='Electronics')
for c in cart:
    print(f"CART ITEM ID: {c.id}, CART ID: {c.cart.id}, PRODUCT ID: {c.product.id}, AMOUNT: {c.amount}")
    #product__categories: คือการเข้าถึงหมวดหมู่ (categories) ของ Product ที่อยู่ใน CartItem (FK)

CART ITEM ID: 1, CART ID: 1, PRODUCT ID: 1, AMOUNT: 1
CART ITEM ID: 76, CART ID: 26, PRODUCT ID: 1, AMOUNT: 3
CART ITEM ID: 157, CART ID: 53, PRODUCT ID: 1, AMOUNT: 3
CART ITEM ID: 175, CART ID: 59, PRODUCT ID: 1, AMOUNT: 2
CART ITEM ID: 238, CART ID: 80, PRODUCT ID: 1, AMOUNT: 1
CART ITEM ID: 273, CART ID: 91, PRODUCT ID: 1, AMOUNT: 1
CART ITEM ID: 370, CART ID: 124, PRODUCT ID: 1, AMOUNT: 2
CART ITEM ID: 451, CART ID: 151, PRODUCT ID: 1, AMOUNT: 3
CART ITEM ID: 453, CART ID: 151, PRODUCT ID: 1, AMOUNT: 2
CART ITEM ID: 477, CART ID: 159, PRODUCT ID: 1, AMOUNT: 1
CART ITEM ID: 19, CART ID: 7, PRODUCT ID: 2, AMOUNT: 2
CART ITEM ID: 48, CART ID: 16, PRODUCT ID: 2, AMOUNT: 3
CART ITEM ID: 177, CART ID: 59, PRODUCT ID: 2, AMOUNT: 1
CART ITEM ID: 223, CART ID: 75, PRODUCT ID: 2, AMOUNT: 4
CART ITEM ID: 225, CART ID: 75, PRODUCT ID: 2, AMOUNT: 2
CART ITEM ID: 243, CART ID: 81, PRODUCT ID: 2, AMOUNT: 1
CART ITEM ID: 305, CART ID: 102, PRODUCT ID: 2, AMOUNT: 1
CART ITEM ID: 371, CART ID: 124, 

CART ITEM ID: 330, CART ID: 110, PRODUCT ID: 19, AMOUNT: 1
CART ITEM ID: 339, CART ID: 113, PRODUCT ID: 19, AMOUNT: 1
CART ITEM ID: 407, CART ID: 136, PRODUCT ID: 19, AMOUNT: 1
CART ITEM ID: 486, CART ID: 162, PRODUCT ID: 19, AMOUNT: 1
CART ITEM ID: 510, CART ID: 170, PRODUCT ID: 19, AMOUNT: 4
CART ITEM ID: 520, CART ID: 174, PRODUCT ID: 19, AMOUNT: 4
CART ITEM ID: 141, CART ID: 47, PRODUCT ID: 20, AMOUNT: 2
CART ITEM ID: 181, CART ID: 61, PRODUCT ID: 20, AMOUNT: 3
CART ITEM ID: 184, CART ID: 62, PRODUCT ID: 20, AMOUNT: 1
CART ITEM ID: 240, CART ID: 80, PRODUCT ID: 20, AMOUNT: 4
CART ITEM ID: 296, CART ID: 99, PRODUCT ID: 20, AMOUNT: 2
CART ITEM ID: 314, CART ID: 105, PRODUCT ID: 20, AMOUNT: 3
CART ITEM ID: 366, CART ID: 122, PRODUCT ID: 20, AMOUNT: 1
CART ITEM ID: 389, CART ID: 130, PRODUCT ID: 20, AMOUNT: 1
CART ITEM ID: 396, CART ID: 132, PRODUCT ID: 20, AMOUNT: 4
CART ITEM ID: 517, CART ID: 173, PRODUCT ID: 20, AMOUNT: 1


In [ ]:
โจทย์ 10: Query หาข้อมูล PaymentMethod ที่ใช้วิธีการชำระเงินแบบ QR

In [89]:
method = PaymentMethod.objects.filter(method="QR")
for p in method:
    print(f"PAYMENT METHOD ID: {p.id}, PAYMENT ID: {p.payment.id}, PRICE: {p.price}")

PAYMENT METHOD ID: 2, PAYMENT ID: 4, PRICE: 8868.22
PAYMENT METHOD ID: 3, PAYMENT ID: 5, PRICE: 579.71
PAYMENT METHOD ID: 4, PAYMENT ID: 6, PRICE: 4104.35
PAYMENT METHOD ID: 6, PAYMENT ID: 8, PRICE: 19897.65
PAYMENT METHOD ID: 8, PAYMENT ID: 10, PRICE: 2822.27
PAYMENT METHOD ID: 10, PAYMENT ID: 12, PRICE: 3371.30
PAYMENT METHOD ID: 12, PAYMENT ID: 14, PRICE: 4146.69
PAYMENT METHOD ID: 13, PAYMENT ID: 15, PRICE: 6001.07
PAYMENT METHOD ID: 20, PAYMENT ID: 22, PRICE: 559.06
PAYMENT METHOD ID: 21, PAYMENT ID: 23, PRICE: 5328.17
PAYMENT METHOD ID: 22, PAYMENT ID: 24, PRICE: 4725.04
PAYMENT METHOD ID: 24, PAYMENT ID: 26, PRICE: 1603.59
PAYMENT METHOD ID: 25, PAYMENT ID: 27, PRICE: 288671.59
PAYMENT METHOD ID: 26, PAYMENT ID: 28, PRICE: 3195.35
PAYMENT METHOD ID: 31, PAYMENT ID: 33, PRICE: 3249.16
PAYMENT METHOD ID: 35, PAYMENT ID: 37, PRICE: 7922.45
PAYMENT METHOD ID: 40, PAYMENT ID: 42, PRICE: 4016.79
PAYMENT METHOD ID: 41, PAYMENT ID: 43, PRICE: 4024.69
PAYMENT METHOD ID: 43, PAYMENT ID: 4

PAYMENT METHOD ID: 2, PAYMENT ID: 4, PRICE: 8868.22
PAYMENT METHOD ID: 3, PAYMENT ID: 5, PRICE: 579.71
PAYMENT METHOD ID: 4, PAYMENT ID: 6, PRICE: 4104.35
PAYMENT METHOD ID: 6, PAYMENT ID: 8, PRICE: 19897.65
PAYMENT METHOD ID: 8, PAYMENT ID: 10, PRICE: 2822.27
PAYMENT METHOD ID: 10, PAYMENT ID: 12, PRICE: 3371.30
PAYMENT METHOD ID: 12, PAYMENT ID: 14, PRICE: 4146.69
PAYMENT METHOD ID: 13, PAYMENT ID: 15, PRICE: 6001.07
PAYMENT METHOD ID: 20, PAYMENT ID: 22, PRICE: 559.06
PAYMENT METHOD ID: 21, PAYMENT ID: 23, PRICE: 5328.17
PAYMENT METHOD ID: 22, PAYMENT ID: 24, PRICE: 4725.04
PAYMENT METHOD ID: 24, PAYMENT ID: 26, PRICE: 1603.59
PAYMENT METHOD ID: 25, PAYMENT ID: 27, PRICE: 288671.59
PAYMENT METHOD ID: 26, PAYMENT ID: 28, PRICE: 3195.35
PAYMENT METHOD ID: 31, PAYMENT ID: 33, PRICE: 3249.16
PAYMENT METHOD ID: 35, PAYMENT ID: 37, PRICE: 7922.45
PAYMENT METHOD ID: 40, PAYMENT ID: 42, PRICE: 4016.79
PAYMENT METHOD ID: 41, PAYMENT ID: 43, PRICE: 4024.69
PAYMENT METHOD ID: 43, PAYMENT ID: 4